In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

# Load the dataset
file_path = r"C:\Users\georg\OneDrive\Desktop\My Ideas\Weathersystem\november_weather_data.csv"
weather_data = pd.read_csv(file_path)

# Preprocess the dataset (convert temperatures to °C, add new features)
weather_data["Max Temperature (°C)"] = (weather_data["Max Temperature (°F)"] - 32) * 5.0 / 9.0
weather_data["Avg Temperature (°C)"] = (weather_data["Avg Temperature (°F)"] - 32) * 5.0 / 9.0
weather_data["Min Temperature (°C)"] = (weather_data["Min Temperature (°F)"] - 32) * 5.0 / 9.0

# Handle missing values in the target variable
weather_data["Avg Temperature (°C)"].fillna(weather_data["Avg Temperature (°C)"].mean(), inplace=True)

# Add new features
weather_data["Temperature Range (°C)"] = weather_data["Max Temperature (°C)"] - weather_data["Min Temperature (°C)"]
weather_data["Humidity_Wind_Interaction"] = weather_data["Avg Humidity (%)"] * weather_data["Avg Wind Speed (mph)"]

# Features and target variable
X = weather_data[["Avg Humidity (%)", "Avg Wind Speed (mph)", "Avg Pressure (in)"]]
y = weather_data["Avg Temperature (°C)"]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestRegressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the trained model
model_file = r"C:\Users\georg\OneDrive\Desktop\My Ideas\Weathersystem\weather_model.pkl"
joblib.dump(model, model_file)
print(f"Model saved to {model_file}")


In [ ]:
import joblib
import requests
import pandas as pd
from time import sleep

# Load the saved model
model_file = r"C:\Users\georg\OneDrive\Desktop\My Ideas\Weathersystem\weather_model.pkl"
model = joblib.load(model_file)

# OpenWeatherMap API details
API_KEY = "f35e1cead7b85a1b309ec04af0fd9338"
CITY_NAME = "Järfälla"
URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY_NAME}&appid={API_KEY}&units=metric"

# Simulated starting indoor temperature
indoor_temp = 18.0  # Example starting point

# Target indoor temperature
TARGET_TEMPERATURE = 20.0

# Function to adjust indoor temperature
def adjust_temperature(indoor_temp, predicted_temp):
    if indoor_temp < TARGET_TEMPERATURE:
        print(f"Current indoor temperature: {indoor_temp}°C")
        print("Temperature is low. Turning ON heating.")
        indoor_temp += 1  # Simulating heating effect
    elif indoor_temp > TARGET_TEMPERATURE:
        print(f"Current indoor temperature: {indoor_temp}°C")
        print("Temperature is high. Turning ON cooling.")
        indoor_temp -= 1  # Simulating cooling effect
    else:
        print(f"Current indoor temperature: {indoor_temp}°C")
        print("Temperature is stable. No action required.")
    return indoor_temp

# Start periodic updates
while True:
    try:
        # Fetch real-time weather data
        response = requests.get(URL, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Extract relevant information
        current_temp = data["main"]["temp"]
        current_humidity = data["main"]["humidity"]
        current_wind_speed = data["wind"]["speed"]
        current_pressure = data["main"]["pressure"]

        # Prepare real-time data for prediction
        real_time_data = pd.DataFrame({
            "Avg Humidity (%)": [current_humidity],
            "Avg Wind Speed (mph)": [current_wind_speed * 2.237],  # Convert m/s to mph
            "Avg Pressure (in)": [current_pressure * 0.02953]  # Convert hPa to inHg
        })

        # Predict the ideal temperature
        predicted_temp = model.predict(real_time_data)[0]

        print("\n--- Temperature Adjustment ---")
        print(f"Outdoor Temperature: {current_temp}°C")
        print(f"Predicted Indoor Temperature: {predicted_temp}°C")

        # Adjust indoor temperature
        indoor_temp = adjust_temperature(indoor_temp, predicted_temp)

        print(f"Updated Indoor Temperature: {indoor_temp}°C")
        print("--- End of Update ---\n")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")

    # Wait for 30 minutes before next update
    sleep(1800)
